<a href="https://colab.research.google.com/github/dnguyend/ManNullRange/blob/master/tests/complex_psd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\rN}{\mathrm{N}}$
$\newcommand{\rD}{\mathrm{D}}$
$\newcommand{\cE}{\mathcal{E}}$
$\newcommand{\cEJ}{\cE_{\JJ}}$
$\newcommand{\CTRL}{\textsc{CTRL}}$
$\newcommand{\Herm}[3]{\mathrm{Sym}_{#1, #2, #3}}$
$\newcommand{\AHerm}[3]{\mathrm{Skew}_{#1, #2, #3}}$
$\newcommand{\St}[3]{\mathrm{St}_{#1, #2, #3}}$
$\newcommand{\Sp}[3]{\mathtt{S}^{+}_{#1, #2, #3}}$
$\newcommand{\Sd}[2]{\mathtt{S}^{+}_{#1, #2}}$
$\DeclareMathOperator{\UU}{U}$
$\DeclareMathOperator{\JJ}{J}$
# Complex Positive-semidefinite manifold $\Sp{\C}{p}{n}$ with Riemannian quotient metric.
* Manifold is Stiefel $\St{\C}{p}{n}$ times positive definite (PD) $\Sd{\C}{p}$ with the quotient by the unitary group $\UU(\C, p)$, via a representation $S = YPY^H$ with $Y\in\St{\C}{p}{n}, P\in \Sd{\C}{p}$.
* Horizontal lift is normal to vertical vectors.
* Ambient space $\cE= \C^{n\times p}\oplus\C^{p\times p}$
* The tangent space is the nullspace of the operator $\JJ(S)$ is $[\omega_P-\omega_P^H,  \alpha_1 Y^{H} \omega_Y+\beta(\omega_P P^{-1}-\omega_P P^{-1}) ]$ in $\cE_{\JJ} = \AHerm{\C}{p}{p}\oplus \C^{p\times p}$
* Operator $\rN(S)$ is $\rN[B, D]=[\beta Y(P^{-1}D - D P^{-1}) + Y_{\perp}B, \alpha_1 D]$ parameterizing the tangent space by $\cE_{\rN} = \C^{(n-p)\times p}\oplus \Herm{H}{\C}{p}$.
* We show an example of optimization, finding the rank $p$ matrix closest to a given matrix $A$.
* We also show a detailed verification the implementation against a number of theoretical considerations: property of projection, the covariant derivatives is metric invariant and torsion free, and relationship between bilinear and operator Riemannian Hessian, various adjoint relations. 

In [1]:
!git clone https://github.com/dnguyend/ManNullRange.git

Cloning into 'ManNullRange'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 137 (delta 90), reused 89 (delta 54), pack-reused 0
Receiving objects: 100% (137/137), 153.03 KiB | 440.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.


In [2]:
!git clone  https://github.com/pymanopt/pymanopt.git
import sys
sys.path.append("/content/pymanopt")



Cloning into 'pymanopt'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 4124 (delta 43), reused 33 (delta 12), pack-reused 4047
Receiving objects: 100% (4124/4124), 907.23 KiB | 1.21 MiB/s, done.
Resolving deltas: 100% (2863/2863), done.


First we import the package. Basic structure: The manifold is ComplexPositiveSemidefinite, a manifold point is by the class psd_point, a point on the tangent space is psd_ambient. We have a few utility functions:
crandn for complex random, and hsym is the Hermitian symmetrize operator $\frac{1}{2}(X+X^H)$. Check zero just checks if a matrix is close to zero.

In [3]:
import numpy as np
import pymanopt
from numpy import trace
from numpy.random import (randint)
from ManNullRange.manifolds.ComplexPositiveSemidefinite import (ComplexPositiveSemidefinite, psd_ambient, psd_point)
from ManNullRange.manifolds.tools import (crandn, hsym)
from ManNullRange.tests.test_tools import check_zero

This function set up to find the shortest distance to a manifold defined by man

In [4]:
def find_shortest_distance(man, A, X0, maxiter, check_deriv=False):
    # simple function. Distance from a given matrix to the manifold
    # || S - A||_F^2
    from pymanopt import Problem
    from pymanopt.solvers import TrustRegions
    from pymanopt.function import Callable

    @Callable
    def cost(S):
        diff = (A - S.Y @ S.P @ S.Y.T.conjugate())
        val = trace(diff @ diff.T.conjugate()).real
        return val

    @Callable
    def egrad(S):
        return psd_ambient(-4*A @ S.Y @ S.P,
                           2*(S.P-S.Y.T.conjugate() @ A @ S.Y))
    @Callable
    def ehess(S, xi):
        return psd_ambient(
            -4*A @ (xi.tY @ S.P + S.Y @ xi.tP),
            2*(xi.tP - xi.tY.T.conjugate() @ A @ S.Y -
               S.Y.T.conjugate() @ A @ xi.tY))
    if check_deriv:
        xi = man.randvec(X0)
        dlt = 1e-7
        S1 = psd_point(X0.Y+dlt*xi.tY, X0.P + dlt*xi.tP)
        print((cost(S1) - cost(X0))/dlt)
        print(man.base_inner_ambient(egrad(X0), xi))
        h1 = egrad(S1) - egrad(X0)
        h1 = psd_ambient(h1.tY/dlt, h1.tP/dlt)
        h2 = ehess(X0, xi)
        print(check_zero(h1.tY - h2.tY) + check_zero(h1.tP - h2.tP))
        return

    prob = Problem(
        man, cost=cost, egrad=egrad, ehess=ehess)

    solver = TrustRegions(maxtime=100000, maxiter=maxiter, use_rand=False)
    opt = solver.solve(prob, x=X0, Delta_bar=250)
    return opt
    

Then generate a $1000\times 1000$ matrix that is close to a positive-semidefinite manifold of rank 20 manifold. Higher rank matrices take  longer time to run.

In [5]:
n, d = (1000, 20)
Y0, _ = np.linalg.qr(crandn(n, d))
P0 = np.diag(randint(1, 100, d)*.01)

def ht(A):
  return A.T.conjugate()

A00 = Y0 @ P0 @ ht(Y0)
A0 = hsym(A00)
A1 = np.diag(crandn(n))*1e-2 + crandn(n, n)*1e-4
# PY = Y0 @ ht(Y0)
# A1 = PY @ A1  + A1 @ PY - PY@ A1 @ PY
A = (hsym(A1) + A0)
alpha = np.array([1, 1])
print("alpha %s" % str(alpha))
beta = alpha[1] * .01
man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
XInit = man.rand()
opt_pre = find_shortest_distance(man, A, X0=XInit, maxiter=30, check_deriv=False)



alpha [1 1]
Optimizing...
                                            f: +5.719428e+04   |grad|: 4.491219e+05
acc       k:     1     num_inner:     2     f: +2.356402e+04   |grad|: 1.770884e+05   reached target residual-kappa (linear)
acc       k:     2     num_inner:     3     f: +9.469439e+03   |grad|: 6.941300e+04   reached target residual-kappa (linear)
acc       k:     3     num_inner:     3     f: +3.806891e+03   |grad|: 2.723159e+04   reached target residual-kappa (linear)
acc       k:     4     num_inner:     2     f: +1.557487e+03   |grad|: 1.073639e+04   reached target residual-kappa (linear)
acc       k:     5     num_inner:     2     f: +6.304780e+02   |grad|: 4.229255e+03   reached target residual-kappa (linear)
acc       k:     6     num_inner:     2     f: +2.538543e+02   |grad|: 1.654801e+03   reached target residual-kappa (linear)
acc       k:     7     num_inner:     2     f: +1.040962e+02   |grad|: 6.472437e+02   reached target residual-kappa (linear)
acc       k:   

We could see the gradient drops fast from 1e5 to single digit. We use this pre optimize version with small beta as initial value to run the next one, with beta=1, then with beta = 30 as we get close to the optimal value:

In [6]:
if True:
  beta = alpha[1] * 1
  man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
  opt_mid = find_shortest_distance(man, A, X0=opt_pre, maxiter=30)


beta = alpha[1] * 30
man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
opt = find_shortest_distance(man, A, X0=opt_mid, maxiter=50)
opt_mat = opt.Y @ opt.P @ opt.Y.T.conjugate()


Optimizing...
                                            f: +2.484441e+00   |grad|: 1.378346e+00
REJ TR-   k:     1     num_inner:     2     f: +2.484441e+00   |grad|: 1.378346e+00   negative curvature
REJ TR-   k:     2     num_inner:     2     f: +2.484441e+00   |grad|: 1.378346e+00   negative curvature
acc TR-   k:     3     num_inner:     1     f: +2.191079e+00   |grad|: 1.905835e+00   exceeded trust region
acc TR+   k:     4     num_inner:     0     f: +1.669840e+00   |grad|: 4.947410e-01   exceeded trust region
acc       k:     5     num_inner:     1     f: +1.566099e+00   |grad|: 7.853107e-01   exceeded trust region
acc TR+   k:     6     num_inner:     2     f: +1.328344e+00   |grad|: 4.718467e-01   exceeded trust region
acc       k:     7     num_inner:     2     f: +1.066747e+00   |grad|: 6.590485e-01   negative curvature
REJ TR-   k:     8     num_inner:     5     f: +1.066747e+00   |grad|: 6.590485e-01   exceeded trust region
acc TR+   k:     9     num_inner:     1     f: 

Check that it recovers the eigenvalues:

In [7]:
from numpy.linalg import eigh
eh_in = eigh(A0)[0]
eh_out = eigh(opt_mat)[0]
print(eh_in[-(d+2):])
print(eh_out[-(d+2):])

[8.10309016e-16 9.12590494e-16 3.00000000e-02 9.00000000e-02
 1.00000000e-01 1.10000000e-01 1.60000000e-01 2.10000000e-01
 2.70000000e-01 2.70000000e-01 2.90000000e-01 3.00000000e-01
 3.40000000e-01 3.40000000e-01 4.50000000e-01 5.80000000e-01
 5.80000000e-01 7.70000000e-01 9.10000000e-01 9.10000000e-01
 9.20000000e-01 9.40000000e-01]
[7.42597583e-16 8.48360427e-16 3.39653955e-02 9.10040622e-02
 1.00384357e-01 1.10712064e-01 1.60949428e-01 2.10790954e-01
 2.69793541e-01 2.70204403e-01 2.89716585e-01 3.00235592e-01
 3.39491130e-01 3.40389557e-01 4.49222517e-01 5.79649296e-01
 5.80022115e-01 7.69816864e-01 9.09806731e-01 9.09978784e-01
 9.19585008e-01 9.40210001e-01]


Our testing strategy is provide a vectorization to test the adjoints, test the various properties of projection is satisfied, and check the covariant derivatives is compatible with metric and is torsion free.

In [10]:
from numpy import zeros, trace, allclose
import numpy.linalg as la

from ManNullRange.manifolds.ComplexPositiveSemidefinite import (
    ComplexPositiveSemidefinite, psd_ambient, psd_point)    
from ManNullRange.manifolds.tools import crandn, hsym, complex_extended_lyapunov


In [13]:
def test_vectorize():
    from ManNullRange.manifolds.tools import cvech, cunvech, cvecah, cunvecah
    p = 10
    mat = crandn(p, p)
    mat += mat.T.conjugate()
    v = cvech(mat)
    mat2 = cunvech(v)
    print(check_zero(mat-mat2))

    # check inner product:
    for i in range(p*p):
        v = zeros(p*p)
        v[i] = 1
        h = cunvech(v)
        assert(np.abs(trace(h @ h.T.conjugate()).real-1) < 1e-10)

    # now do skew
    mat = crandn(p, p)
    mat -= mat.T.conjugate()
    v = cvecah(mat)
    mat2 = cunvecah(v)
    print(check_zero(mat-mat2))
    
    for i in range(p*p):
        v = zeros(p*p)
        v[i] = 1
        h = cunvecah(v)
        assert(np.abs(trace(h @ h.T.conjugate()).real-1) < 1e-10)
        
for i in range(3):
  test_vectorize()

4.440892098500626e-16
2.482534153247273e-16
2.2887833992611187e-16
4.440892098500626e-16
3.1401849173675503e-16
4.440892098500626e-16


Note that vectorization above is just a convenient function for testing. We tested that vec and unvec are inverse functions, and an orthogonal basis on the vectorized correspond to an orthogonal basis under the Frobenius inner product. In the next block we check the inner product, and check the implementation of $J$

In [14]:
def test_inner(man, S):
    for i in range(10):
        alpha = man.alpha
        bt = man.beta
        eta1 = man._rand_ambient()
        eta2 = man._rand_ambient()
        Y = S.Y
        Pinv = S.Pinv
        inn1 = alpha[0]*trace(eta1.tY.T.conjugate() @ eta2.tY) +\
            (alpha[1]-alpha[0])*trace((eta1.tY.T.conjugate()@Y) @
                                      (Y.T.conjugate() @ eta2.tY)) +\
            bt*trace(Pinv@eta1.tP@Pinv@eta2.tP.T.conjugate())
        assert(allclose(man.inner(S, eta1, eta2), inn1.real))
        
        eta2a = man.g_inv(S, eta2)
        v1 = man.inner(S, eta1, eta2a)
        v2 = trace(eta1.tY @ eta2.tY.T.conjugate()) +\
            trace(eta1.tP@eta2.tP.T.conjugate())
        assert(allclose(v1, v2.real))
    print(True)


def test_J(man, S):
    al = man.alpha
    bt = man.beta
    
    def diff_i(ii):
        U = zeros(man.tdim_St+man.tdim_P)
        U[ii] = 1
        E = man._unvec(U)
        jE = man.J(S, E)
        jE1 = {'P': E.tP - E.tP.T.conjugate(),
               'YP': al[1]*S.Y.T.conjugate()@E.tY +
               bt*(E.tP@S.Pinv-S.Pinv@E.tP)}
        return np.mean(np.abs(man._vec_range_J(jE)-man._vec_range_J(jE1)))

    diffs = zeros(man.tdim_St + man.tdim_P)
    for ii in range(diffs.shape[0]):
        diffs[ii] = diff_i(ii)
    try:
        assert(allclose(diffs, 0))
        print(True)
    except Exception:
        print(False)

alpha = randint(1, 10, 2) * .1
beta = randint(1, 10, 1)[0] * .02
n = 5
d = 3
man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
S = man.rand()

test_inner(man, S)
test_J(man, S)



True
True


We then check the adjoint $Jst$, by vectorizing, representing $J$ as a matrix and take adjoint, then compare with the implementation

In [15]:

def make_j_mat(man, S):
    # philopsophy is if there is a direct sum then cvec a component at a time
    # so vec

    codim = man.codim
    ret = zeros((codim, man.tdim_P + man.tdim_St))
    for ii in range(ret.shape[1]):
        ee = np.zeros(ret.shape[1])
        ee[ii] = 1            
        ret[:, ii] = man._vec_range_J(
            man.J(S, man._unvec(ee)))
    return ret
        

def make_g_inv_mat(man, S):
    nn = man.tdim_P + man.tdim_St
    ret = zeros((nn, nn))
    for ii in range(ret.shape[1]):
        ee = np.zeros(ret.shape[1])
        ee[ii] = 1            
        eeS = man._unvec(ee)
        ret[:, ii] = man._vec(
            man.g_inv(S, eeS))
    return ret

def test_Jst(man, S, jmat):
    print("test JST")
    for ii in range(10):
        a = man._rand_range_J()
        avec = man._vec_range_J(a)
        jtout = jmat.T @ avec
        jtout2 = man._vec(man.Jst(S, a))
        # print(np.where(np.abs(jtout - jtout2) > 1e-9))
        diff = check_zero(jtout-jtout2)
        print(diff)

jmat = make_j_mat(man, S)
test_Jst(man, S, jmat)    

test JST
1.7763568394002505e-15
8.881784197001252e-16
1.7763568394002505e-15
8.881784197001252e-16
1.7763568394002505e-15
1.7763568394002505e-15
4.440892098500626e-16
8.881784197001252e-16
1.7763568394002505e-15
4.440892098500626e-16


We then test the extended Lyapunov equation:

In [16]:
def test_lyapunov():
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 1)[0] * .02
    n = 5
    d = 3
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
    S = man.rand()

    P = S.P
    B = crandn(d, d)
    alpha1 = alpha[1]
    
    def L(X, P):
        Piv = la.inv(P)
        return (alpha1 - 2*beta)*X + beta*(P@X@Piv + Piv@X@P)
    X = complex_extended_lyapunov(alpha1, beta, P, B)
    # L(X, P)
    print(check_zero(B-L(X, P)))

for i in range(10):
  test_lyapunov()

2.4245857620654024e-14
2.2991268696100664e-14
4.318467880798754e-15
1.1801832636420706e-15
1.1631705747361533e-15
3.5614848888819304e-15
9.155133597044475e-16
4.902690697762858e-15
4.906130692364006e-14
1.2412670766236366e-15


Test implementation of $g^{-1}J^H$. We represent the metric as an operator, take its inverse, then compute $g^{-1}J^H$ in two different ways and compare

In [17]:
  ginv_mat = make_g_inv_mat(man, S)
  ee = man._rand_ambient()
  g1 = man._unvec(ginv_mat @ man._vec(ee))
  print(check_zero(man.g_inv(S, ee).tP-g1.tP))
  print(check_zero(man.g_inv(S, ee).tY-g1.tY))
  # test g_inv_Jst
  for ii in range(10):
      a = man._rand_range_J()
      avec = man._vec_range_J(a)
      jtout = ginv_mat @ jmat.T @ avec
      jtout2 = man._vec(man.g_inv_Jst(S, a))
      diff = check_zero(jtout-jtout2)
      print(diff)


1.224444715065716e-12
2.0380985511143445e-15
5.4569682106375694e-12
7.275957614183426e-12
1.4551915228366852e-11
7.275957614183426e-12
1.8189894035458565e-12
7.275957614183426e-12
7.275957614183426e-12
2.9103830456733704e-11
7.275957614183426e-12
1.4551915228366852e-11


Test projection in two ways, using the $J$ operator approach and the $N$ operator as in the paper. 
We check: 
* projection is horizonal, 
* the stiefel part of projection satisfies the condition for tangent vector on Stiefel and the PD part is symmetric.
* $\langle\omega, \eta\rangle = \langle \Pi\omega, \eta\rangle$ if $\eta$ is in the tangent space and $\omega$ in ambient space.
* The "N" method is same as the "J" method:

In [18]:

    
def test_projection(man, S):
    print("test projection")
    N = 10
    ret = np.zeros(N)

    for i in range(N):
        U = man._rand_ambient()
        Upr = man.proj(S, U)
        ret[i] = check_zero(man._vec_range_J(man.J(S, Upr)))
    print(ret)
    
    if check_zero(ret) > 1e-8:
        print("not all works")
    else:
        print("All good")
    print("test randvec")
    ret_pr = np.zeros(N)
    for i in range(N):
        XX = man.rand()
        H = man.randvec(XX)
        U = man._rand_ambient()
        Upr = man.proj(XX, U)
        ret_pr[i] = man.inner(XX, U, H) - man.inner(XX, Upr, H)
        ret[i] = (check_zero(XX.Y.T.conjugate() @ H.tY +
                             H.tY.T.conjugate() @ XX.Y))
    print(ret)
    if check_zero(ret) > 1e-9:
        print("not all works")
    else:
        print("All good")
    print("check inner of projection = inner of original")
    print(ret_pr)
    if check_zero(ret_pr) > 1e-8:
        print("not all works")
    else:
        print("All good")



def test_N_proj():
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 1)[0] * .02
    n = 10
    d = 6
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
    S = man.rand()
    """
    def proj_range_alt(man, S, U):
        # projection. U is in ambient
        # return one in tangent
        al1 = man.alpha[1]
        beta = man.beta
        YTU = S.Y.T@U.tY
        D0 = sym(U.tP + YTU@S.P - S.P@YTU)
        D = _extended_lyapunov(al1, beta, S.P, D0, S.evl, S.evec)
        return psd_ambient(
            beta*S.Y@(S.Pinv@D-D@S.Pinv) + U.tY - S.Y@(S.Y.T@U.tY), al1*D)
    """
    U = man.randvec(S)
    Upr1 = super(ComplexPositiveSemidefinite, man).proj(S, U)
    Upr2 = man.proj(S, U)
    Upr3 = man.proj_range_alt(S, U)
    print(check_zero(Upr1.tP-Upr2.tP))
    print(check_zero(Upr1.tY-Upr2.tY))
    print(check_zero(Upr1.tY-Upr3.tY))
    print(check_zero(Upr1.tP-Upr3.tP))
test_projection(man, S)
test_N_proj()

test projection
[7.10542736e-16 3.30346745e-15 6.28036983e-16 2.51214793e-15
 1.25607397e-15 5.02429587e-15 1.25607397e-15 2.51214793e-15
 1.47659662e-15 9.76996262e-16]
All good
test randvec
[5.57880165e-16 6.75357949e-16 1.33226763e-15 8.55033921e-16
 2.63677968e-16 3.97399365e-16 9.08995101e-16 6.24500451e-16
 3.35371896e-16 6.52256027e-16]
All good
check inner of projection = inner of original
[-9.99200722e-16  0.00000000e+00  1.11022302e-16  1.11022302e-16
  2.22044605e-16 -2.22044605e-16 -1.11022302e-16  0.00000000e+00
 -6.66133815e-16  3.13638004e-15]
All good
2.144915255004782e-15
1.8670653977139892e-16
4.665728978696896e-16
2.458749249743224e-14


Test the various expression related to inverting $Jg^{-1}J^H$:

In [19]:
    for i in range(10):
        a = man._rand_range_J()
        eta = man._rand_ambient()
        print(man.base_inner_ambient(eta, man.Jst(S, a)))
        print((trace((eta.tP.T.conjugate() - eta.tP) @ a['P'] + (
            man.alpha[1]*eta.tY.T.conjugate() @ S.Y + man.beta*(
                S.Pinv @ eta.tP.T.conjugate() -
                eta.tP.T.conjugate() @ S.Pinv)) @ a['YP'])).real)

        print((trace(2 * eta.tP.T.conjugate() @ a['P']) + trace((
            man.alpha[1]*eta.tY.T.conjugate() @ S.Y + man.beta*(
                S.Pinv @ eta.tP.T.conjugate() -
                eta.tP.T.conjugate() @ S.Pinv)) @ a['YP'])).real)

        print((trace(eta.tP.T.conjugate() @ (
            2*a['P'] + man.beta*(a['YP'] @ S.Pinv - S.Pinv @ a['YP'])))
              + trace(eta.tY.T.conjugate() @
                      (man.alpha[1] * S.Y @ a['YP']))).real)

        print(man.base_inner_E_J(man.J(S, eta), a))
        print((trace((eta.tP - eta.tP.T.conjugate()).T.conjugate() @ a['P'] + (
            man.alpha[1]*S.Y.T.conjugate() @ eta.tY + man.beta*(
                eta.tP @ S.Pinv - S.Pinv @ eta.tP)).T.conjugate() @
                a['YP'])).real)

    for i in range(10):
        a = man._rand_range_J()
        beta = man.beta
        alf = man.alpha
        anew1 = man.J(S, man.g_inv_Jst(S, a))
        
        anew = {}
        saYP = a['YP'] + a['YP'].T.conjugate()
        anew['P'] = 4/beta * S.P @ a['P'] @ S.P + S.P @ saYP - saYP @ S.P
        anew['YP'] = alf[1]*a['YP'] + beta*(
            ((2/man.beta)*S.P@a['P']@S.P + S.P @ a['YP'] - a['YP'] @ S.P) @
            S.Pinv - S.Pinv @ ((2/man.beta)*S.P@a['P']@S.P + S.P @ a['YP'] -
                               a['YP'] @ S.P))
        
        anew['YP'] = alf[1]*a['YP'] + (
            (2*S.P@a['P'] + beta*S.P @ a['YP'] @ S.Pinv - beta*a['YP'])
            - (2*a['P']@S.P + beta*a['YP'] - beta*S.Pinv@a['YP'] @ S.P))

        anew['YP'] = (alf[1]-2*beta)*a['YP'] + (
            (2*S.P@a['P'] + beta*S.P @ a['YP'] @ S.Pinv)
            - (2*a['P']@S.P - beta*S.Pinv@a['YP'] @ S.P))

        anew['YP'] = (alf[1]-2*beta)*a['YP'] + (
            (2*S.P@a['P'] - 2*a['P']@S.P + beta*S.P @ a['YP'] @ S.Pinv +
             beta*S.Pinv@a['YP'] @ S.P))
        print(check_zero(man._vec_range_J(anew1)-man._vec_range_J(anew)))

    for i in range(10):
        a = man._rand_range_J()
        b1 = man.J(S, man.g_inv_Jst(S, a))
        b2 = man.J_g_inv_Jst(S, a)
        print(
            check_zero(man._vec_range_J(b1)-man._vec_range_J(b2)))
        a1 = man.solve_J_g_inv_Jst(S, b1)
        print(check_zero(
            man._vec_range_J(a)-man._vec_range_J(a1)))
                
    for ii in range(10):
        E = man._rand_ambient()
        a2 = man.J_g_inv(S, E)
        a1 = man.J(S, man.g_inv(S, E))
        print(check_zero(man._vec_range_J(a1)-man._vec_range_J(a2)))
    
    for i in range(20):
        Uran = man._rand_ambient()
        Upr = man.proj(S, man.g_inv(S, Uran))
        Upr2 = man.proj_g_inv(S, Uran)
        print(check_zero(man._vec(Upr)-man._vec(Upr2)))



18.25498800672762
18.25498800672762
18.254988006727622
18.25498800672762
18.25498800672762
18.25498800672762
1.0971487117398104
1.0971487117398082
1.09714871173981
1.0971487117398104
1.0971487117398082
1.0971487117398082
-7.57863494066776
-7.578634940667761
-7.57863494066776
-7.57863494066776
-7.5786349406677616
-7.57863494066776
22.507611918497844
22.507611918497837
22.507611918497844
22.507611918497844
22.50761191849784
22.507611918497837
7.852561556176931
7.852561556176931
7.85256155617693
7.852561556176931
7.852561556176932
7.852561556176931
47.42716358980119
47.42716358980118
47.42716358980119
47.42716358980119
47.42716358980118
47.42716358980118
15.42728192775585
15.427281927755848
15.427281927755846
15.42728192775585
15.427281927755848
15.427281927755848
9.039737880774002
9.039737880774
9.039737880774
9.039737880774002
9.039737880774
9.039737880774
-13.954800184168155
-13.95480018416816
-13.954800184168157
-13.954800184168155
-13.954800184168159
-13.95480018416816
42.23317751445

Check numerical derivatives of $J$, $Jst$, and metric $g$:

In [20]:
    for ii in range(10):
        a = man._rand_range_J()
        xi = man.randvec(S)
        jtout2 = man.Jst(S, a)
        dlt = 1e-7
        Snew = psd_point(S.Y + dlt*xi.tY, S.P+dlt*xi.tP)
        jtout2a = man.Jst(Snew, a)
        d1 = (jtout2a - jtout2).scalar_mul(1/dlt)
        d2 = man.D_Jst(S, xi, a)
        print(check_zero(man._vec(d2)-man._vec(d1)))

    for ii in range(10):
        S1 = man.rand()
        eta = man._rand_ambient()
        xi = man.randvec(S1)
        a1 = man.J(S1, eta)
        dlt = 1e-8
        Snew = psd_point(S1.Y + dlt*xi.tY, S1.P+dlt*xi.tP)
        a2 = man.J(Snew, eta)
        d1 = {'P': (a2['P']-a1['P'])/dlt,
              'YP': (a2['YP']-a1['YP'])/dlt}
        d2 = man.D_J(S1, xi, eta)
        print(check_zero(man._vec_range_J(d2) - man._vec_range_J(d1)))
                
    # derives metrics
    for ii in range(10):
        S1 = man.rand()
        xi = man.randvec(S1)
        omg1 = man._rand_ambient()
        omg2 = man._rand_ambient()
        dlt = 1e-7
        S2 = psd_point(S1.Y + dlt*xi.tY, S1.P+dlt*xi.tP)
        p1 = man.inner(S1, omg1, omg2)
        p2 = man.inner(S2, omg1, omg2)
        der1 = (p2-p1)/dlt
        der2 = man.base_inner_ambient(
            man.D_g(S1, xi, omg2), omg1)
        print(der1-der2)
        if np.abs(der1 - der2) > 1e-4:
            print(der1, der2)
            break



5.422492219686603e-07
1.3671869463216524e-06
1.158077353569098e-06
2.1873056255117262e-06
4.7339066755469617e-07
1.1091370347160456e-07
3.406762495439253e-07
8.231435182359803e-07
1.0224067705788542e-06
5.385345178687828e-07
3.888399924961705e-09
9.356830998896726e-08
1.6150875886689064e-08
8.241860700863857e-09
2.521897386476013e-07
4.54905429297936e-09
1.894326570606175e-08
1.0303641218012416e-08
1.4326659608654424e-08
9.31851436986042e-09
-1.1593208948390554e-07
8.211570357019582e-08
-5.3495068588205186e-06
8.683065719217176e-08
3.3700477564124753e-07
-1.0252358539908357e-07
-2.5187529004710996e-06
1.101837026062924e-07
3.0934905739243845e-06
-3.6661672697668735e-08


Derivatives of projection D_proj. Check it against numerical derivatives:

In [21]:
    for i in range(1):
        e = man._rand_ambient()
        S1 = man.rand()
        xi = man.randvec(S1)
        dlt = 1e-7
        S2 = psd_point(S1.Y + dlt*xi.tY, S1.P+dlt*xi.tP)

        # S = psd_point(S1.Y, S1.P)
        d1P = (man.proj(S2, e).tP - man.proj(S1, e).tP)/dlt
        d1Y = (man.proj(S2, e).tY - man.proj(S1, e).tY)/dlt
        d2 = man.D_proj(S1, xi, e)
        print(check_zero(d1P-d2.tP) + check_zero(d1Y-d2.tY))
    


5.920157540046997e-07


Check the crossterm satisfies $<x12, xi>$ is same as $<D_g(xi, eta1), eta2>$
Check Christoffel term $K$ is symmetric and double check the formula:



In [22]:
    # cross term for Christoffel:    
    for i in range(10):
        S1 = man.rand()
        xi = man.randvec(S1)
        eta1 = man.randvec(S1)
        eta2 = man.randvec(S1)
        dr1 = man.D_g(S1, xi, eta1)
        x12 = man.contract_D_g(S1, eta1, eta2)

        p1 = man.base_inner_ambient(dr1, eta2)
        p2 = man.base_inner_ambient(x12, xi)
        # print(p1, p2, p1-p2)
        print(p1-p2)

    # now test christofel:
    # two things: symmetric on vector fields
    # and christofel relation
    # in the case metric

    for i in range(10):
        S1 = man.rand()
        xi = man.randvec(S1)
        eta1 = man.randvec(S1)
        eta2 = man.randvec(S1)
        p1 = man.proj_g_inv(S1, man.christoffel_form(S1, xi, eta1))
        p2 = man.proj_g_inv(S1, man.christoffel_form(S1, eta1, xi))
        print(check_zero(man._vec(p1)-man._vec(p2)))
        v1 = man.base_inner_ambient(
            man.christoffel_form(S1, eta1, eta2), xi)
        v2 = man.base_inner_ambient(man.D_g(S1, eta1, eta2), xi)
        v3 = man.base_inner_ambient(man.D_g(S1, eta2, eta1), xi)
        v4 = man.base_inner_ambient(man.D_g(S1, xi, eta1), eta2)
        # print(v1, 0.5*(v2+v3-v4), v1-0.5*(v2+v3-v4))
        print(v1-0.5*(v2+v3-v4))


8.326672684688674e-17
6.938893903907228e-18
-1.5829351718288365e-17
-2.42861286636753e-17
-6.938893903907228e-18
-6.661338147750939e-16
2.7755575615628914e-17
-1.0842021724855044e-17
0.0
1.2434497875801753e-14
0.0
-6.245004513516506e-17
0.0
0.0
0.0
2.0816681711721685e-17
0.0
6.938893903907228e-18
0.0
2.040034807748725e-15
0.0
-1.3877787807814457e-17
0.0
-5.551115123125783e-17
0.0
-1.3877787807814457e-17
0.0
0.0
0.0
-3.469446951953614e-17


Check preserving metric on flat space:

In [25]:
def test_christ_flat():
    """now test that christofel preserves metrics:
    on the flat space
    d_xi <v M v> = 2 <v M nabla_xi v>
     v = proj(W) @ (aa W + b)
    """
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 2)[0] * .1
    n = 20
    d = 3
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
    S = man.rand()
    
    xi = man.randvec(S)
    xi = man.randvec(S)
    aa = crandn(n*d, n*d)
    bb = crandn(n*d)
    cc = crandn(d*d, d*d)
    dd = hsym(crandn(d, d))
        
    def v_func_flat(S):
        # a function from the manifold
        # to ambient
        csp = hsym((cc @ S.P.reshape(-1)).reshape(d, d))
        
        return psd_ambient(
            (aa @ S.Y.reshape(-1) + bb).reshape(n, d),
            csp + dd)

    vv = v_func_flat(S)
    dlt = 1e-7
    Snew = psd_point(
        S.Y + dlt * xi.tY,
        S.P + dlt * xi.tP)
    vnew = v_func_flat(Snew)

    val = man.inner(S, vv)
    valnew = man.inner(Snew, vnew)
    d1 = (valnew - val)/dlt
    dv = (vnew - vv).scalar_mul(1/dlt)
    nabla_xi_v = dv + man.g_inv(
        S, man.christoffel_form(S, xi, vv))
    nabla_xi_va = dv + man.g_inv(
        S, super(ComplexPositiveSemidefinite, man).christoffel_form(S, xi, vv))
    print(check_zero(man._vec(nabla_xi_v) - man._vec(nabla_xi_va)))
    d2 = man.inner(S, vv, nabla_xi_v)

    print(d1)
    print(2*d2)

test_christ_flat()

0.0
-5.961805982224178
-5.961813781985541


metric compatibility

In [26]:

def calc_covar_numeric(man, S, xi, v_func):
    """ compute nabla on E dont do the metric
    lower index. So basically
    Nabla (Pi e).
    Thus, if we want to do Nabla Pi g_inv df
    We need to send g_inv df
    """

    def vv_func(W):
        return man.proj(W, v_func(W))
    
    vv = vv_func(S)

    dlt = 1e-7
    Snew = psd_point(S.Y + dlt*xi.tY, S.P + dlt * xi.tP)
    vnew = vv_func(Snew)

    val = man.inner(S, vv)
    valnew = man.inner(Snew, vnew)
    d1 = (valnew - val)/dlt
    dv = (vnew - vv).scalar_mul(1/dlt)
    cx = man.christoffel_form(S, xi, vv)
    nabla_xi_v_up = dv + man.g_inv(S, cx)
    nabla_xi_v = man.proj(S, nabla_xi_v_up)
    
    if False:
        d2 = man.inner(S, vv, nabla_xi_v)
        d2up = man.inner(S, vv, nabla_xi_v_up)

        print(d1)
        print(2*d2up)
        print(2*d2)
    return nabla_xi_v, dv, cx


def test_chris_vectorfields():
    # now test that it works on embedded metrics
    # we test that D_xi (eta g eta) = 2(eta g nabla_xi eta)
    n, d = (20, 3)
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 1)[0] * .1
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)

    S0 = man.rand()
    aa = crandn(n*d, n*d)
    intc = crandn(n*d)
    cc = crandn(d*d, d*d)
    p_intc = hsym(crandn(d, d))

    inct_xi = man._rand_ambient()
    aa_xi = crandn(n*d, n*d)
    cc_xi = crandn(d*d, d*d)
    
    def v_func(S):
        # a function from the manifold
        # to ambient
        csp = hsym((cc @ (S.P-S0.P).reshape(-1)).reshape(d, d))
        
        return man.proj(S, psd_ambient(
            (aa @ (S.Y-S0.Y).reshape(-1) + intc).reshape(n, d),
            csp + p_intc))

    SS = psd_point(S0.Y, S0.P)
    xi = man.proj(SS, inct_xi)

    nabla_xi_v, dv, cxv = calc_covar_numeric(
        man, SS, xi, v_func)

    def xi_func(S):
        csp_xi = hsym((cc_xi @ (S.P-S0.P).reshape(-1)).reshape(d, d))
        xi_amb = psd_ambient(
            (aa_xi @ (S.Y-S0.Y).reshape(-1) +
             inct_xi.tY.reshape(-1)).reshape(n, d),
            csp_xi + inct_xi.tP)
        return man.proj(S, xi_amb)

    vv = v_func(SS)

    nabla_v_xi, dxi, cxxi = calc_covar_numeric(
        man, SS, vv, xi_func)
    diff = nabla_xi_v - nabla_v_xi
    # print(diff.tY, diff.tP)
    # now do Lie bracket:
    dlt = 1e-7
    SnewXi = psd_point(SS.Y+dlt*xi.tY, SS.P+dlt*xi.tP)
    Snewvv = psd_point(SS.Y+dlt*vv.tY, SS.P+dlt*vv.tP)
    vnewxi = v_func(SnewXi)
    xnewv = xi_func(Snewvv)
    dxiv = (vnewxi - vv).scalar_mul(1/dlt)
    dvxi = (xnewv - xi).scalar_mul(1/dlt)
    diff2 = man.proj(SS, dxiv-dvxi)
    print(check_zero(man._vec(diff) - man._vec(diff2)))
test_chris_vectorfields()

3.645428225951264e-08


Check $\Pi(D_{\xi}\imath\eta - D_{\eta}\imath\xi) = \imath[\xi, \eta]$, check covariant derivatives given by the formula is the covariant derivatives of the gradient:

In [27]:
                            

def num_deriv_amb(man, S, xi, func, dlt=1e-7):
    Snew = psd_point(S.Y + dlt*xi.tY,
                     S.P + dlt*xi.tP)
    return (func(Snew) - func(S)).scalar_mul(1/dlt)

    
def test_covariance_deriv():
    # now test full:
    # do covariant derivatives
    # check that it works, preseving everything
    n, d = (5, 3)
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 2)[0] * .01
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)
    S = man.rand()
    
    aa = crandn(n*d, n*d)
    cc = crandn(d*d, d*d)
    icpt = man._rand_ambient()

    def omg_func(S):
        csp = hsym((cc @ S.P.reshape(-1)).reshape(d, d))
        return psd_ambient(
            (aa @ S.Y.reshape(-1) + icpt.tY.reshape(-1)).reshape(n, d),
            csp + icpt.tP)

    xi = man.randvec(S)
    egrad = omg_func(S)
    ecsp = hsym((cc @ xi.tP.reshape(-1)).reshape(d, d))
    ehess = psd_ambient(
        (aa @ xi.tY.reshape(-1)).reshape(n, d),
        ecsp)

    val1 = man.ehess2rhess(S, egrad, ehess, xi)

    def rgrad_func(W):
        return man.proj_g_inv(W, omg_func(W))

    if False:
        first = ehess
        a = man.J_g_inv(S, egrad)
        rgrad = man.proj_g_inv(S, egrad)
        second = man.D_g(
            S, xi, man.g_inv(S, egrad)).scalar_mul(-1)
        aout = man.solve_J_g_inv_Jst(S, a)
        third = man.proj(S, man.D_g_inv_Jst(S, xi, aout)).scalar_mul(-1)
        fourth = man.christoffel_form(S, xi, rgrad)
        val1a1 = man.proj_g_inv(S, first + second + fourth) + third
        print(check_zero(man._vec(val1-val1a1)))
    elif False:
        d_xi_rgrad = num_deriv_amb(man, S, xi, rgrad_func)
        rgrad = man.proj_g_inv(S, egrad)
        fourth = man.christoffel_form(S, xi, rgrad)
        val1a = man.proj(S, d_xi_rgrad) + man.proj_g_inv(S, fourth)
        print(check_zero(man._vec(val1-val1a)))

    # nabla_v_xi, dxi, cxxi
    val2a, _, _ = calc_covar_numeric(man, S, xi, omg_func)
    val2, _, _ = calc_covar_numeric(man, S, xi, rgrad_func)
    # val2_p = project(prj, val2)
    val2_p = man.proj(S, val2)
    # print(val1)
    # print(val2_p)
    print(check_zero(man._vec(val1)-man._vec(val2_p)))
    if True:
        H = xi
        valrangeA_ = ehess + man.g(S, man.D_proj(
            S, H, man.g_inv(S, egrad))) - man.D_g(
                S, H, man.g_inv(S, egrad)) +\
            man.christoffel_form(S, H, man.proj_g_inv(S, egrad))
        valrangeB = man.proj_g_inv(S, valrangeA_)
    valrange = man.ehess2rhess_alt(S, egrad, ehess, xi)
    print(check_zero(man._vec(valrange)-man._vec(val2_p)))
    print(check_zero(man._vec(valrange)-man._vec(val1)))
    print(check_zero(man._vec(valrange)-man._vec(valrangeB)))

test_covariance_deriv()    


0.0015903865350992419
0.0015903865387372207
6.366462912410498e-12
0.0


In [29]:
def test_rhess_02():
    n, d = (50, 3)
    alpha = randint(1, 10, 2) * .1
    beta = randint(1, 10, 2)[0] * .1
    man = ComplexPositiveSemidefinite(n, d, alpha=alpha, beta=beta)

    S = man.rand()
    # simple function. Distance to a given matrix
    # || S - A||_F^2
    A = hsym(crandn(n, n))

    def f(S):
        diff = (A - S.Y @ S.P @ S.Y.T.conjugate())
        return trace(diff @ diff.T.conjugate())

    def df(S):
        return psd_ambient(-4*A @ S.Y @ S.P,
                           2*(S.P-S.Y.T.conjugate() @ A @ S.Y))

    def ehess_form(S, xi, eta):
        return (trace(-4*A @ (xi.tY @ S.P + S.Y @ xi.tP) @
                      eta.tY.T.conjugate()) +
                2*trace((xi.tP - xi.tY.T.conjugate()@A@S.Y -
                         S.Y.T.conjugate()@A@xi.tY) @
                        eta.tP.T.conjugate())).real

    def ehess_vec(S, xi):
        return psd_ambient(-4*A @ (xi.tY @ S.P + S.Y @ xi.tP),
                           2*(xi.tP - xi.tY.T.conjugate() @ A @ S.Y -
                              S.Y.T.conjugate() @ A @ xi.tY))

    xxi = man.randvec(S)
    dlt = 1e-8
    Snew = psd_point(
        S.Y+dlt*xxi.tY, S.P + dlt*xxi.tP)
    d1 = (f(Snew) - f(S))/dlt
    d2 = df(S)
    print("difference of numerical vs analytic directional derivatives of function:")
    print(d1 - man.base_inner_ambient(d2,  xxi))

    eeta = man.randvec(S)

    d1 = man.base_inner_ambient((df(Snew) - df(S)), eeta) / dlt
    ehess_val = ehess_form(S, xxi, eeta)
    dv2 = ehess_vec(S, xxi)
    print("Eucliean Hessian bilinear form, 2 ways")
    print(man.base_inner_ambient(dv2, eeta))
    print(d1, ehess_val, d1-ehess_val)

    # now check the formula: ehess = xi (eta_func(f)) - <D_xi eta, df(Y)>
    # promote eta to a vector field.

    m1 = crandn(n, n)
    m2 = crandn(d, d)
    m_p = crandn(d*d, d*d)

    def eta_field(Sin):
        return man.proj(S, psd_ambient(
            m1 @ (Sin.Y - S.Y) @ m2,
            hsym((m_p @ (Sin.P - S.P).reshape(-1)).reshape(d, d)))) + eeta

    # xietaf: should go to ehess(xi, eta) + df(Y) @ etafield)
    xietaf = (man.base_inner_ambient(df(Snew), eta_field(Snew)) -
              man.base_inner_ambient(df(S), eta_field(S))) / dlt
    # appy eta_func to f: should go to tr(m1 @ xxi @ m2 @ df(Y).T)
    Dxietaf = man.base_inner_ambient(
        (eta_field(Snew) - eta_field(S)), df(S))/dlt
    # this is ehess. should be same as d1 or ehess_val
    # print(xietaf-Dxietaf)
    print("Check the proposition on Euclidean Hessian")
    print(check_zero(xietaf-Dxietaf-ehess_val))

    # now check: rhess. Need to make sure xi, eta in the tangent space.
    # first compare this with numerical differentiation
    xi1 = man.proj(S, xxi)
    eta1 = man.proj(S, eeta)
    egvec = df(S)
    ehvec = ehess_vec(S, xi1)
    rhessvec = man.ehess2rhess(S, egvec, ehvec, xi1)

    # check it numerically:
    def rgrad_func(Y):
        return man.proj_g_inv(Y, df(Y))
    
    # val2a, _, _ = calc_covar_numeric_raw(man, W, xi1, df)
    val2, _, _ = calc_covar_numeric(man, S, xi1, rgrad_func)
    val2_p = man.proj(S, val2)
    # print(rhessvec)
    # print(val2_p)
    print("check numerical covariant derivative of gradient is same as rhess implemeted")
    print(check_zero(man._vec(rhessvec-val2_p)))
    rhessval = man.inner(S, rhessvec, eta1)
    print(man.inner(S, val2, eta1))
    print(rhessval)

    # check symmetric:
    ehvec_e = ehess_vec(S, eta1)
    rhessvec_e = man.ehess2rhess(S, egvec, ehvec_e, eta1)
    rhessval_e = man.inner(S, rhessvec_e, xi1)
    print(rhessval_e)
    # the above computed inner_prod(Nabla_xi Pi * df, eta)
    # in the following check. Extend eta1 to eta_proj
    # (Pi Nabla_hat Pi g_inv df, g eta)
    # = D_xi (Pi g_inv df, g eta) - (Pi g_inv df g Pi Nabla_hat eta)
    
    def eta_proj(S):
        return man.proj(S, eta_field(S))
    print(check_zero(man._vec(eta1-eta_proj(S))))
    
    e1 = man.inner(S, man.proj_g_inv(S, df(S)), eta_proj(S))
    e1a = man.base_inner_ambient(df(S), eta_proj(S))
    print(e1, e1a, e1-e1a)
    Snew = psd_point(S.Y + dlt*xi1.tY, S.P + dlt*xi1.tP)
    e2 = man.inner(Snew, man.proj_g_inv(Snew, df(Snew)), eta_proj(Snew))
    e2a = man.base_inner_ambient(df(Snew), eta_proj(Snew))
    print(e2, e2a, e2-e2a)
    
    first = (e2 - e1)/dlt
    first1 = (man.base_inner_ambient(df(Snew), eta_proj(Snew)) -
              man.base_inner_ambient(df(S), eta_proj(S)))/dlt
    print(first-first1)
    
    val3, _, _ = calc_covar_numeric(man, S, xi1, eta_proj)
    second = man.inner(S, man.proj_g_inv(S, df(S)), man.proj(S, val3))
    second2 = man.inner(S, man.proj_g_inv(S, df(S)), val3)
    print(second, second2, second-second2)
    print('same as rhess_val %f' % (first-second))
    

test_rhess_02()

difference of numerical vs analytic directional derivatives of function:
(1.8886406738971573e-05+0j)
Eucliean Hessian bilinear form, 2 ways
8.203484782198368
8.203484331102201 8.203484782198359 -4.510961577608441e-07
Check the proposition on Euclidean Hessian
3.590739652281627e-07
check numerical covariant derivative of gradient is same as rhess implemeted
0.0007808632510659663
6.941969595476252
6.941966576729185
6.941966576729251
0.0
1.0563740500333503 1.0563740500334704 -1.2012613126444194e-13
1.0563708738383393 1.0563708738383422 -2.886579864025407e-15
1.1723955140041653e-05
-324.56148018465757 -324.56148018465757 0.0
same as rhess_val 6.941979
